In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# loading linear algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# tree based algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# machine learning processing and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve


In [2]:
# pandas view options
pd.set_option('display.max_rows', None)

In [14]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [15]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [16]:
# drop the EIN columns
df = application_df.drop(['EIN'], axis = 1)
df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# number of unique values in each column
print(df.nunique())

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [18]:
# number of data points per unique value in each column
for col in df.columns:
    print(col)
    print(df[col].nunique())
    print(df[col].value_counts())
    print()

NAME
19568
PARENT BOOSTER USA INC                                                    1260
TOPS CLUB INC                                                              765
UNITED STATES BOWLING CONGRESS INC                                         700
WASHINGTON STATE UNIVERSITY                                                492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                            408
PTA TEXAS CONGRESS                                                         368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                              331
ALPHA PHI SIGMA                                                            313
TOASTMASTERS INTERNATIONAL                                                 293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                          287
LITTLE LEAGUE BASEBALL INC                                                 277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                   266
MOMS CLUB                                

In [19]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Prof Booth
replace = df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = [x for x in replace.keys() if replace[x] < 10]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Look at CLASSIFICATION value counts for binning
df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = df["CLASSIFICATION"].value_counts()
classifications_to_replace = [x for x in cutoff.keys() if cutoff[x] < 10]

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [22]:
df.NAME.value_counts()

PARENT BOOSTER USA INC                                                    1260
TOPS CLUB INC                                                              765
UNITED STATES BOWLING CONGRESS INC                                         700
WASHINGTON STATE UNIVERSITY                                                492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                            408
PTA TEXAS CONGRESS                                                         368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                              331
ALPHA PHI SIGMA                                                            313
TOASTMASTERS INTERNATIONAL                                                 293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                          287
LITTLE LEAGUE BASEBALL INC                                                 277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                   266
MOMS CLUB                                           

In [24]:
# Choose a cutoff value and create a list of names to be replaced

name_cutoff = df["NAME"].value_counts()
names_to_replace = [x for x in name_cutoff.keys() if name_cutoff[x] < 2]

# Replace in dataframe
for name in names_to_replace:
    df['NAME'] = df['NAME'].replace(name,"Other")
    
# Check to make sure binning was successful
df['NAME'].value_counts()

Other                                                                     18776
PARENT BOOSTER USA INC                                                     1260
TOPS CLUB INC                                                               765
UNITED STATES BOWLING CONGRESS INC                                          700
WASHINGTON STATE UNIVERSITY                                                 492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                             408
PTA TEXAS CONGRESS                                                          368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                               331
ALPHA PHI SIGMA                                                             313
TOASTMASTERS INTERNATIONAL                                                  293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                           287
LITTLE LEAGUE BASEBALL INC                                                  277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
df_oh = pd.get_dummies(df)
df_oh.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [26]:
# drop smaller half of binary
df_oh.drop('SPECIAL_CONSIDERATIONS_Y', axis=1, inplace=True)

In [13]:
# # Split our preprocessed data into our features and target arrays
# X = df_oh.drop(["IS_SUCCESSFUL"], axis=1).values
# y = df_oh["IS_SUCCESSFUL"].values

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.25, stratify=y)

# print(len(X_train) / len(X))
# print(len(X_test) / len(X))

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Scale onlt the ASK_AMT column Prof Booth
df_oh["ASK_AMT"] = scaler.fit_transform(df_oh["ASK_AMT"].values.reshape(-1, 1))
df_oh.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
0,1,-0.031725,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,-0.030536,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,1,-0.031725,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1,-0.031706,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,1,-0.030146,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [28]:
# Split our preprocessed data into our features and target arrays
X = df_oh.drop(["IS_SUCCESSFUL"], axis=1).values
y = df_oh["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.25, stratify=y)

print(len(X_train) / len(X))
print(len(X_test) / len(X))

0.7499927111577597
0.25000728884224027


## Compile, Train and Evaluate the Model

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                8640      
                                                                 
 dense_9 (Dense)             (None, 8)                 88        
                                                                 
 dense_10 (Dense)            (None, 8)                 72        
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 8,809
Trainable params: 8,809
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [37]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5032 - accuracy: 0.7619
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4286 - accuracy: 0.8010
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4187 - accuracy: 0.8047
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4130 - accuracy: 0.8087
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4098 - accuracy: 0.8071
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4087 - accuracy: 0.8091
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4066 - accuracy: 0.8089
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4046 - accuracy: 0.8098
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4033 - accuracy: 0.8103
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4031 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.3875 - accuracy: 0.8161
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3873 - accuracy: 0.8176
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3868 - accuracy: 0.8181
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3872 - accuracy: 0.8165
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3871 - accuracy: 0.8168
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3867 - accuracy: 0.8176
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3868 - accuracy: 0.8168
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3867 - accuracy: 0.8160
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3868 - accuracy: 0.8167
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.3869 - accuracy: 

In [38]:
# # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4550 - accuracy: 0.7976 - 244ms/epoch - 910us/step
Loss: 0.45498478412628174, Accuracy: 0.7975510358810425


In [34]:
# Export our model to HDF5 file

nn.save('AlphabetSoupCharity_optimization.h5')